# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [3]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sales_train_validation.csv')

In [4]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOUSEHOLD')]

In [6]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks/List of Product ID according to 4 demand patterns/California/Household/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

In [7]:
sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [8]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [9]:
def Croston(ts,extra_periods=1,alpha=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), periodicity(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
    q = 1 #periods since last demand observation
    
    # Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1 + first_occurence
    f[0] = a[0]/p[0]
    # Create all the t+1 forecasts
    for t in range(0,cols):        
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = alpha*q + (1-alpha)*p[t]
            f[t+1] = a[t+1]/p[t+1]
            q = 1           
        else:
            a[t+1] = a[t]
            p[t+1] = p[t]
            f[t+1] = f[t]
            q += 1
       
    # Future Forecast 
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# User-defined functions to calculate Metrics and Croston algorithm

# Set value for parameters

In [10]:
list_params = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]

In [11]:
list_params

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [12]:
start_train_date = '2014-04-11'
end_train_date = '2016-04-10'
n_pred_days = 14
num_train_needed = 50
validation_training_ratio = 0.95

# Pipeline for Croston

In [13]:
def Croston_output_all_params(sales_pattern, list_params, start_train_date, end_train_date, 
                       n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
    train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]
    
    df_result_all_params = pd.DataFrame()
    for product in list_pattern_py:
        train_first = train_data_first_part[product]
        train_second = train_data_second_part[product]
        for i in list_params:
            predictions = list()
            history = [x for x in train_first[num_train_needed*-1:]]
            for t in range(len(train_second)):
                yhat = Croston(history, extra_periods=1, alpha=i)['Forecast'].iloc[-1]
                predictions.append(yhat)
                history.append(train_second[t])
            df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                            'Actual Data': train_second,
                                            'Forecast': predictions,
                                            'Alpha': [i for count in range(len(train_second))]})
            df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)
            
    return df_result_all_params

In [14]:
start = timer()

pattern_df_result_all_params = Croston_output_all_params(sales_erratic,
                                                      list_params,
                                                      start_train_date,
                                                      end_train_date, 
                                                      n_pred_days,
                                                      num_train_needed,
                                                      validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.15581542195000112 minutes


In [15]:
pattern_df_result_all_params

,Product,Actual Data,Forecast,Alpha
0,HOUSEHOLD_1_173_CA_1_validation,3.0,5.425928,0.1
1,HOUSEHOLD_1_173_CA_1_validation,16.0,5.204127,0.1
2,HOUSEHOLD_1_173_CA_1_validation,8.0,6.199721,0.1
3,HOUSEHOLD_1_173_CA_1_validation,2.0,6.367044,0.1
4,HOUSEHOLD_1_173_CA_1_validation,1.0,5.958273,0.1
...,...,...,...,...
13315,HOUSEHOLD_1_151_CA_4_validation,2.0,1.907885,0.9
13316,HOUSEHOLD_1_151_CA_4_validation,7.0,1.977610,0.9
13317,HOUSEHOLD_1_151_CA_4_validation,1.0,6.413915,0.9
13318,HOUSEHOLD_1_151_CA_4_validation,4.0,1.550583,0.9


In [16]:
pattern_df_result_all_params.to_csv('Croston_Erratic_All_Parameters_Hyperparameter_Tuning_Data.csv')

# Summay all metrics

In [17]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [18]:
start = timer()

df_result_metrics_all_params = pattern_df_result_all_params.groupby(['Product', 'Alpha']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.014546337916666137 minutes


In [19]:
df_result_metrics_all_params

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_024_CA_3_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0
1,HOUSEHOLD_1_024_CA_3_validation,0.2,NaN,NaN,NaN,NaN,0.0,0.0
2,HOUSEHOLD_1_024_CA_3_validation,0.3,NaN,NaN,NaN,NaN,0.0,0.0
3,HOUSEHOLD_1_024_CA_3_validation,0.4,NaN,NaN,NaN,NaN,0.0,0.0
4,HOUSEHOLD_1_024_CA_3_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...
355,HOUSEHOLD_2_499_CA_2_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
356,HOUSEHOLD_2_499_CA_2_validation,0.6,NaN,NaN,NaN,NaN,0.0,0.0
357,HOUSEHOLD_2_499_CA_2_validation,0.7,NaN,NaN,NaN,NaN,0.0,0.0
358,HOUSEHOLD_2_499_CA_2_validation,0.8,NaN,NaN,NaN,NaN,0.0,0.0


# Check what products has unexpected metrics result

In [20]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_024_CA_3_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0
1,HOUSEHOLD_1_024_CA_3_validation,0.2,NaN,NaN,NaN,NaN,0.0,0.0
2,HOUSEHOLD_1_024_CA_3_validation,0.3,NaN,NaN,NaN,NaN,0.0,0.0
3,HOUSEHOLD_1_024_CA_3_validation,0.4,NaN,NaN,NaN,NaN,0.0,0.0
4,HOUSEHOLD_1_024_CA_3_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...
355,HOUSEHOLD_2_499_CA_2_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
356,HOUSEHOLD_2_499_CA_2_validation,0.6,NaN,NaN,NaN,NaN,0.0,0.0
357,HOUSEHOLD_2_499_CA_2_validation,0.7,NaN,NaN,NaN,NaN,0.0,0.0
358,HOUSEHOLD_2_499_CA_2_validation,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [21]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_024_CA_3_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0
1,HOUSEHOLD_1_024_CA_3_validation,0.2,NaN,NaN,NaN,NaN,0.0,0.0
2,HOUSEHOLD_1_024_CA_3_validation,0.3,NaN,NaN,NaN,NaN,0.0,0.0
3,HOUSEHOLD_1_024_CA_3_validation,0.4,NaN,NaN,NaN,NaN,0.0,0.0
4,HOUSEHOLD_1_024_CA_3_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...
355,HOUSEHOLD_2_499_CA_2_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
356,HOUSEHOLD_2_499_CA_2_validation,0.6,NaN,NaN,NaN,NaN,0.0,0.0
357,HOUSEHOLD_2_499_CA_2_validation,0.7,NaN,NaN,NaN,NaN,0.0,0.0
358,HOUSEHOLD_2_499_CA_2_validation,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [22]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_024_CA_3_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0
1,HOUSEHOLD_1_024_CA_3_validation,0.2,NaN,NaN,NaN,NaN,0.0,0.0
2,HOUSEHOLD_1_024_CA_3_validation,0.3,NaN,NaN,NaN,NaN,0.0,0.0
3,HOUSEHOLD_1_024_CA_3_validation,0.4,NaN,NaN,NaN,NaN,0.0,0.0
4,HOUSEHOLD_1_024_CA_3_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...
355,HOUSEHOLD_2_499_CA_2_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
356,HOUSEHOLD_2_499_CA_2_validation,0.6,NaN,NaN,NaN,NaN,0.0,0.0
357,HOUSEHOLD_2_499_CA_2_validation,0.7,NaN,NaN,NaN,NaN,0.0,0.0
358,HOUSEHOLD_2_499_CA_2_validation,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [23]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_024_CA_3_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0
1,HOUSEHOLD_1_024_CA_3_validation,0.2,NaN,NaN,NaN,NaN,0.0,0.0
2,HOUSEHOLD_1_024_CA_3_validation,0.3,NaN,NaN,NaN,NaN,0.0,0.0
3,HOUSEHOLD_1_024_CA_3_validation,0.4,NaN,NaN,NaN,NaN,0.0,0.0
4,HOUSEHOLD_1_024_CA_3_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...
355,HOUSEHOLD_2_499_CA_2_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
356,HOUSEHOLD_2_499_CA_2_validation,0.6,NaN,NaN,NaN,NaN,0.0,0.0
357,HOUSEHOLD_2_499_CA_2_validation,0.7,NaN,NaN,NaN,NaN,0.0,0.0
358,HOUSEHOLD_2_499_CA_2_validation,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [24]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [25]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [26]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [27]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [28]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 17.5 %
Percentage of unexpected values of WMAPE is: 17.5 %
Percentage of unexpected values of SMAPE is: 20.0 %
Percentage of unexpected values of MAPE is: 17.5 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [29]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [30]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.1,0.849637,68.168644,75.981678,71.065081,3.576105,4.680416
0.2,0.867285,69.789169,76.901356,72.334662,3.622810,4.718674
0.3,0.888727,71.671100,78.010950,74.397166,3.694514,4.789977
0.4,0.908807,73.419647,79.055437,76.401753,3.764554,4.868713
0.5,0.926148,74.968450,80.008337,78.202653,3.822559,4.952539
0.6,0.944898,76.609109,81.162241,80.145084,3.894677,5.045015
0.7,0.964623,78.320809,82.448566,82.208078,3.978642,5.151484
0.8,0.988023,80.345491,84.029072,84.657894,4.078234,5.278298
0.9,1.014723,82.642256,85.816853,87.420106,4.192347,5.433206


In [31]:
for mts in list_metrics:
    print('The optimum Alpha based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha based on Mean MASE is: 0.1
The optimum Alpha based on Mean WMAPE is: 0.1
The optimum Alpha based on Mean SMAPE is: 0.1
The optimum Alpha based on Mean MAPE is: 0.1
The optimum Alpha based on Mean MAE is: 0.1
The optimum Alpha based on Mean RMSE is: 0.1


### Get MEDIAN metrics value of each Alpha

In [32]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.1,0.826857,64.48885,72.71020,72.65195,2.835402,3.576850
0.2,0.809131,66.37290,73.96930,74.48470,2.928307,3.746174
0.3,0.820720,68.31115,75.44150,77.72430,3.064799,3.884284
0.4,0.837825,70.15995,76.29120,79.34930,3.179638,3.997834
0.5,0.858317,71.77280,76.64865,79.80235,3.230508,4.039208
0.6,0.886107,73.62505,77.81695,80.51420,3.187220,4.025212
0.7,0.896585,75.56445,79.27020,81.92055,3.125693,4.025993
0.8,0.932226,76.89395,81.79830,84.37415,3.154695,4.134286
0.9,0.959960,77.55440,84.21705,88.01220,3.296716,4.266756


In [33]:
for mts in list_metrics:
    print('The optimum Alpha based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha based on Median of MASE is: 0.2
The optimum Alpha based on Median of WMAPE is: 0.1
The optimum Alpha based on Median of SMAPE is: 0.1
The optimum Alpha based on Median of MAPE is: 0.1
The optimum Alpha based on Median of MAE is: 0.1
The optimum Alpha based on Median of RMSE is: 0.1


# Run Croston model after decide best paramaters

In [34]:
best_alpha = 0.1

In [35]:
def Croston_test_data(best_alpha, sales_pattern, start_train_date, end_train_date, 
                       n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    
    df_result_best_params = pd.DataFrame()
    for product in list_pattern_py:
        train_product = train_data[product]
        test_product = test_data[product]
        predictions = list()
        history = [x for x in train_product[num_train_needed*-1:]]
        for t in range(len(test_product)):
            yhat = Croston(history, extra_periods=1, alpha=best_alpha)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(test_product[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                        'Actual Data': test_product,
                                        'Forecast': predictions})
        df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)
            
    return df_result_best_params

In [36]:
start = timer()

pattern_df_result_best_params = Croston_test_data(best_alpha, sales_erratic, start_train_date, end_train_date, n_pred_days, num_train_needed, validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.0071986350333342365 minutes


In [37]:
df_result_final = pattern_df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [38]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_024_CA_3_validation,NaN,NaN,NaN,NaN,0.000000,0.000000
1,HOUSEHOLD_1_106_CA_2_validation,NaN,NaN,NaN,NaN,0.000000,0.000000
2,HOUSEHOLD_1_118_CA_2_validation,1.434526,119.8609,130.4355,114.5502,4.965666,5.845035
3,HOUSEHOLD_1_147_CA_2_validation,0.756442,45.2572,51.7974,50.9058,1.454696,1.713874
4,HOUSEHOLD_1_151_CA_2_validation,0.706732,59.4606,62.8680,74.7765,2.718199,3.376117
5,HOUSEHOLD_1_151_CA_4_validation,0.638931,61.7507,63.1196,72.4734,1.720198,2.466531
6,HOUSEHOLD_1_173_CA_1_validation,0.748580,56.7300,65.0959,83.5641,3.282236,5.071646
7,HOUSEHOLD_1_173_CA_2_validation,1.021317,83.9382,99.2999,39.3179,2.278323,2.684400
8,HOUSEHOLD_1_179_CA_1_validation,0.708688,62.0102,71.6708,44.2368,1.417376,1.726948
9,HOUSEHOLD_1_204_CA_1_validation,0.813265,80.4328,82.2779,106.5134,2.815148,4.844163


In [39]:
df_result_final.isnull().sum()

Product    0
MASE       6
WMAPE      6
SMAPE      7
MAPE       6
MAE        0
RMSE       0
dtype: int64

In [40]:
df_result_final.to_csv('Croston_Erratic_Test_Data.csv')